<a href="https://colab.research.google.com/github/lokeshbaranwal/Pose-Estimation-of-Sattelite/blob/master/speed1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json
import tensorflow
import os
#import utils2
#from utils2 import KerasDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
#from submission import SubmissionWriter
from tensorflow.keras.utils import Sequence
#from tensorflow.keras.preprocessing import image as keras_image




In [0]:
class KerasDataGenerator(Sequence):

        """ DataGenerator for Keras to be used with fit_generator (https://keras.io/models/sequential/#fit_generator)"""

        def __init__(self, preprocessor, label_list, speed_root, batch_size=32, dim=(224, 224), n_channels=3, shuffle=True):

            # loading dataset
            self.image_root = os.path.join(speed_root, 'images', 'train')

            # Initialization
            self.preprocessor = preprocessor
            self.dim = dim
            self.batch_size = batch_size
            self.labels = self.labels = {label['filename']: {'q': label['q_vbs2tango'], 'r': label['r_Vo2To_vbs_true']}
                                         for label in label_list}
            self.list_IDs = [label['filename'] for label in label_list]
            self.n_channels = n_channels
            self.shuffle = shuffle
            self.indexes = None
            self.on_epoch_end()

        def __len__(self):

            """ Denotes the number of batches per epoch. """

            return int(np.floor(len(self.list_IDs) / self.batch_size))

        def __getitem__(self, index):

            """ Generate one batch of data """

            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

            # Find list of IDs
            list_IDs_temp = [self.list_IDs[k] for k in indexes]

            # Generate data
            X, y = self.__data_generation(list_IDs_temp)

            return X, y

        def on_epoch_end(self):

            """ Updates indexes after each epoch """

            self.indexes = np.arange(len(self.list_IDs))
            if self.shuffle:
                np.random.shuffle(self.indexes)

        def __data_generation(self, list_IDs_temp):

            """ Generates data containing batch_size samples """

            # Initialization
            X = np.empty((self.batch_size, *self.dim, self.n_channels))
            y = np.empty((self.batch_size, 7), dtype=float)

            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                img_path = os.path.join(self.image_root, ID)
                img =image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                x = self.preprocessor(x)
                X[i,] = x

                q, r = self.labels[ID]['q'], self.labels[ID]['r']
                y[i] = np.concatenate([q, r])

            return X, y


In [0]:
dataset_root="/content/gdrive/My Drive/speed"
dataset='train'



In [0]:
def evaluate(model, dataset, append_submission, dataset_root):

    """ Running evaluation on test set, appending results to a submission. """

    with open(os.path.join(dataset_root, dataset + '.json'), 'r') as f:
        image_list = json.load(f)

    print('Running evaluation on {} set...'.format(dataset))

    for img in image_list:
        img_path = os.path.join(dataset_root, 'images', dataset, img['filename'])
        pil_img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(pil_img)
        x = preprocess_input(x)
        x = np.expand_dims(x, 0)
        output = model.predict(x) 
        append_submission(img['filename'], output[0, :4], output[0, 4:])



In [0]:
params = {'dim': (224, 224),
              'batch_size': 32,
              'n_channels': 3,
              'shuffle': True}

    # Loading and splitting dataset
with open(os.path.join(dataset_root, 'train' + '.json'), 'r') as f:
    label_list = json.load(f)
train_labels = label_list[:int(len(label_list)*.8)]
validation_labels = label_list[int(len(label_list)*.8):]

    # Data generators for training and validation
training_generator = KerasDataGenerator(preprocess_input, train_labels, dataset_root, **params)
validation_generator = KerasDataGenerator(preprocess_input, validation_labels, dataset_root, **params)

    # Loading and freezing pre-trained model
tensorflow.keras.backend.set_learning_phase(0)
pretrained_model = tensorflow.keras.applications.ResNet50(weights="imagenet", include_top=False,
                                                              input_shape=(224, 224, 3))

    # Adding new trainable hidden and output layers to the model
tensorflow.keras.backend.set_learning_phase(1)
x = pretrained_model.output
x = tensorflow.keras.layers.Flatten()(x)
x = tensorflow.keras.layers.Dense(1024, activation="relu")(x)
predictions = tensorflow.keras.layers.Dense(7, activation="linear")(x)
model_final = tensorflow.keras.models.Model(inputs=pretrained_model.input, outputs=predictions)
model_final.compile(loss="mean_squared_error", optimizer='adam')

    # Training the model (transfer learning)
history = model_final.fit_generator(
    training_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[tensorflow.keras.callbacks.ProgbarLogger(count_mode='steps')])

print('Training losses: ', history.history['loss'])
print('Validation losses: ', history.history['val_loss'])


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/20
Epoch 1/20
211/300 [====================>.........] - ETA: 27:18 - loss: 1050.9154

In [0]:
model_final.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model_final.save('speedv1.h5')